In [1]:
import sys
sys.path.append("/home/adam/projects/FreeCAD/build/Mod/Path")
import area

In [2]:
from OCC.Core.BRepTools import breptools_Read
from OCC.Core.TopoDS import *
from OCC.Core.BRep import BRep_Builder
from OCC.Core.gp import *
from OCC.Core.BRepBuilderAPI import *
from math import pi,degrees,radians
from OCC.Core.BRepTools import breptools_Write
import OCCUtils
import OCCUtils.edge
import numpy as np
import OCCUtils.face
from OCC.Core.BRepAlgoAPI import *
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.BRepAdaptor import *
import HackerCAD

from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

In [3]:
file_shape = TopoDS_Shape()
builder = BRep_Builder()
assert breptools_Read(file_shape,"./chamfered_nut.brep",builder)

In [16]:
renderer = JupyterRenderer()

In [17]:
if "aligned_shape" in locals():
    s = aligned_shape
else:
    s = file_shape
for face in OCCUtils.Topo(s).faces():
    renderer.DisplayShape(face)

In [18]:
renderer

## Align the part as it is aligned on the machine

In [7]:
# Select y+ most face (green is y)
face = renderer.GetSelectedShape()
face

ba = BRepAdaptor_Surface(face)

pln = ba.Plane()

pnt = pln.Location()

#  model on machine - where the model is in CAD:
dy = 29.643 - pnt.Y()
print("dy:{}".format(dy))

trsf = gp_Trsf()
trsf.SetTranslation(gp_Vec(0,dy,0))
mt = BRepBuilderAPI_Transform(s,trsf)
aligned_shape = mt.Shape()

dy:29.643


In [24]:
breptools_Write(aligned_shape,"./chamfered_nut_aligned.brep")

True

## Export stl of the aligned shape to load in the [PocketNC simulator](https://sim.pocketnc.com/)

In [13]:
# export stl of the aligned shape for use in 
from OCC.Core.StlAPI import StlAPI_Writer
stl_export = StlAPI_Writer()
stl_export.Write(aligned_shape, "./sim_shape.stl")

True

# Do Adaptive on a face

In [19]:
# select a face to do an adaptive pocketing operation
face = renderer.GetSelectedShape()
face

<class 'TopoDS_Face'>

In [20]:
a2d = HackerCAD.Adaptive2d(face)
a2d.step_over_factor = 0.5
a2d.tool_diameter = 25.4 / 32.0
a2d.helix_diameter = a2d.tool_diameter * 0.5
a2d.helix_angle = 60.0
a2d.flip_ax_dir()
a2d.tolerance = 1e-3
a2d.stockToLeave = 0.0
a2d.z_clearance = 4.0
a2d.z_link_clear = 0.5
a2d.cut_depths = [1.0,0.0]

In [21]:
wire = a2d.compute()

len(a2d_output): 1
helix center point
helix center point


In [22]:
renderer.DisplayShape(wire)

In [11]:
breptools_Write(a2d.comp,"/tmp/bla.brep")

True

In [12]:
pp = HackerCAD.PostProcessor()

In [13]:
motions_gcode = ""
for motions in a2d.motions:
    motions_gcode += pp.process(motions)

In [14]:
gcode = ""
gcode += "G90 G21 G54\n"
gcode += "M6 T1 G43 H1\n"
gcode += "G93\n"
gcode += "M3 S21390\n"
gcode +=  motions_gcode
gcode += "G94\n"
gcode += "G91 G0 Z10\n"
gcode += "G90\n"
gcode += "M5\n"
gcode += "M2\n"

f = open("/tmp/bla.ngc","w")
f.write(gcode)
f.close()

# Try to use adaptive to finish largest ID

In [49]:
# select the step face inside
face = renderer.GetSelectedShape()
face

<class 'TopoDS_Face'>

In [50]:
bac = BRepAdaptor_Surface(face)

In [51]:
pln = bac.Plane()

In [52]:
od_edge = None
max_r = 0.0
for edge in OCCUtils.Topo(face).edges():
    c = BRepAdaptor_Curve(edge)
    circ = c.Circle()
    r = circ.Radius()
    if r > max_r:
        od_edge = edge
        max_r = r
    print(c.FirstParameter(),c.LastParameter(),circ.Radius())
print("selected edge with radius = {}".format(max_r))
    

0.0 6.28318530717959 6.3999999999999995
0.0 6.28318530717959 8.149999999999999
selected edge with radius = 8.149999999999999


In [53]:
mw = BRepBuilderAPI_MakeWire()
mw.Add(od_edge)
wire = mw.Wire()

mf = BRepBuilderAPI_MakeFace(wire)
adaptive_face = mf.Face()

In [54]:
renderer.DisplayShape(adaptive_face)

In [14]:
adaptive_face

<class 'TopoDS_Face'>

In [136]:
wire

<class 'TopoDS_Wire'>

In [14]:
#renderer.DisplayShape(wire)

In [55]:
a2d = HackerCAD.Adaptive2d(adaptive_face)

In [56]:
a2d.step_over_factor = 0.15

In [57]:
a2d.tool_diameter = 25.4 / 16.0

In [58]:
a2d.helix_diameter = a2d.tool_diameter * 0.75

In [59]:
a2d.helix_angle = 60.0

In [60]:
a2d.flip_ax_dir()

In [61]:
a2d.z_lift_distance = 0.0

In [62]:
a2d.tolerance = 1e-3

In [63]:
wire = a2d.compute()

helix center point
skipped a point because |v_next - v_now| <= 1e-9
x:0.53, y:7.327
skipped a point because |v_next - v_now| <= 1e-9
x:0.515, y:7.327
skipped a point because |v_next - v_now| <= 1e-9
x:0.339, y:7.337
skipped a point because |v_next - v_now| <= 1e-9
x:0.182, y:7.343
skipped a point because |v_next - v_now| <= 1e-9
x:-0.594, y:7.32
skipped a point because |v_next - v_now| <= 1e-9
x:-0.767, y:7.303
skipped a point because |v_next - v_now| <= 1e-9
x:-3.14, y:-6.64
skipped a point because |v_next - v_now| <= 1e-9
x:-2.73, y:-6.819
skipped a point because |v_next - v_now| <= 1e-9
x:-2.67, y:-6.844
skipped a point because |v_next - v_now| <= 1e-9
x:0.532, y:-7.325
skipped a point because |v_next - v_now| <= 1e-9
x:0.562, y:-7.323
skipped a point because |v_next - v_now| <= 1e-9
x:6.42, y:-3.567
skipped a point because |v_next - v_now| <= 1e-9
x:6.512, y:-3.396
skipped a point because |v_next - v_now| <= 1e-9
x:6.523, y:-3.374
skipped a point because |v_next - v_now| <= 1e-9
x:

In [64]:
renderer.DisplayShape(wire)

In [65]:
pp = HackerCAD.PostProcessor()

In [66]:
gcode = ""
gcode += "G90 G21 G55\n"
gcode += "M6 T1 G43 H1\n"
gcode += "G93\n"
gcode += pp.process(a2d.motions)
gcode += "G94\n"
gcode += "M2\n"

In [67]:
f = open("/tmp/bla.ngc","w")
f.write(gcode)
f.close()

# Create a toolpath to finish the largest ID

In [21]:
# select axial face inside the id in renderer
face = renderer.GetSelectedShape()

In [22]:
ren = JupyterRenderer()

In [23]:
for edge in OCCUtils.Topo(face).edges():
    ren.DisplayShape(edge)

In [24]:
ren

In [25]:
# select the outside edge
outer_edge = ren.GetSelectedShape()
outer_edge

<class 'TopoDS_Edge'>

In [26]:
# select the inside edge
inner_edge = ren.GetSelectedShape()
inner_edge

<class 'TopoDS_Edge'>

In [27]:
# Making the circle that will be the tool tip path
bac = BRepAdaptor_Curve(outer_edge)
circ = bac.Circle()

bac = BRepAdaptor_Curve(inner_edge)
circ_inner = bac.Circle()

tool_diameter = 25.4 / 8.0
axis = gp_Ax2(circ.Location(),circ.Axis().Reversed().Direction(),circ.XAxis().Direction())
new_circ = gp_Circ(axis,circ.Radius()-tool_diameter/2)
me = BRepBuilderAPI_MakeEdge(new_circ)
tool_tip_edge = me.Edge()

# corresponding shank path
trsf = gp_Trsf()
trsf.SetTranslation(gp_Vec(0,1,0))
mt = BRepBuilderAPI_Transform(tool_tip_edge,trsf)
tool_shank_edge = mt.Shape()

In [28]:
circ.Radius()*2

17.299999999999997

In [29]:
circ.Radius() - circ_inner.Radius()

0.5

In [30]:
rm = HackerCAD.RuledMotion(tool_tip_edge,tool_shank_edge)
rm.feedrate = 1000

In [31]:
renderer.DisplayShape(tool_tip_edge)

In [32]:
# making lead - in
bac_i = BRepAdaptor_Curve(tool_tip_edge)
p1 = bac_i.Value(0)
p0 = gp_Pnt(0,p1.Y(),0)
me = BRepBuilderAPI_MakeEdge(p0,p1)
lead_in_edge = me.Edge()

# corresponding shank path
trsf = gp_Trsf()
trsf.SetTranslation(gp_Vec(0,1,0))
mt = BRepBuilderAPI_Transform(lead_in_edge,trsf)
lead_in_shank_edge = mt.Shape()

In [33]:
renderer.DisplayShape(lead_in_edge)

In [34]:
rm_lead = HackerCAD.RuledMotion(lead_in_edge,lead_in_shank_edge)
rm_lead.feedrate = 10

In [76]:
pp = HackerCAD.PostProcessor()

In [114]:
rm_gcode = pp.process([rm_lead,rm])

In [115]:
gcode = ""
gcode += "G90 G21 G54\n"
gcode += "M6 T1 G43 H1\n"
gcode += "G93\n"
gcode += "M3 S21390\n"
gcode += rm_gcode
gcode += "G94\n"
gcode += "G0 X0 Y0\n"
gcode += "G0 Z40\n"
gcode += "M5\n"
gcode += "M2\n"

In [116]:
f = open("/tmp/bla.ngc","w")
f.write(gcode)
f.close()

# Chamfers
## ID chamfer

In [12]:
# select face to chamfer
chamfer_face = renderer.GetSelectedShape()
chamfer_face

<class 'TopoDS_Face'>

In [13]:
ren = JupyterRenderer()

In [14]:
for edge in OCCUtils.Topo(chamfer_face).edges():
    ren.DisplayShape(edge)

In [15]:
ren

In [16]:
# select tool tip edge
tip_edge = ren.GetSelectedShape()
tip_edge

<class 'TopoDS_Edge'>

In [17]:
# select tool shank edge 
shank_edge = ren.GetSelectedShape()
shank_edge

<class 'TopoDS_Edge'>

In [30]:
rm_chamfer_1 = HackerCAD.RuledMotion(tip_edge,shank_edge)
rm_chamfer_1.feedrate = 1000
rm_chamfer_1.surface_normal_offset = 25.4 / 16.0
rm_chamfer_1.tool_axis_offset = 1.0

In [35]:
# creating a lead in
lead_in_distance = 10.0
rm_chamfer_1.surface_normal_offset += lead_in_distance
v_tip_0,v_shank_0 = rm_chamfer_1.evaluate(0)

rm_chamfer_1.surface_normal_offset -= lead_in_distance
v_tip_1,v_shank_1 = rm_chamfer_1.evaluate(0)

me = BRepBuilderAPI_MakeEdge(
    gp_Pnt(v_tip_0.XYZ()),
    gp_Pnt(v_tip_1.XYZ()))
tip_lead_in_edge = me.Edge()

trsf = gp_Trsf()
trsf.SetTranslation(v_shank_0)

mt = BRepBuilderAPI_Transform(tip_lead_in_edge,trsf)
shank_lead_in_edge = mt.Shape()

In [37]:
lead_motion = HackerCAD.RuledMotion(tip_lead_in_edge,shank_lead_in_edge)
lead_motion.feedrate = 100

In [21]:
pp = HackerCAD.PostProcessor()

In [38]:
rm_gcode = pp.process([lead_motion,rm_chamfer_1])

In [39]:
gcode = ""
gcode += "G90 G21 G54\n"
gcode += "M6 T1 G43 H1\n"
gcode += "G93\n"
gcode += "M3 S21390\n"
gcode += rm_gcode
gcode += "G94\n"
gcode += "G91 G0 Y10\n"
gcode += "G90\n"
gcode += "G0 Z40\n"
gcode += "M5\n"
gcode += "M2\n"

In [40]:
f = open("/tmp/bla.ngc","w")
f.write(gcode)
f.close()

## OD chamfer

In [18]:
# select face to chamfer
chamfer_face = renderer.GetSelectedShape()
chamfer_face

<class 'TopoDS_Face'>

In [19]:
ren = JupyterRenderer()

In [20]:
for edge in OCCUtils.Topo(chamfer_face).edges():
    ren.DisplayShape(edge)

In [21]:
ren

In [22]:
# select tool tip edge (outside edge)
tip_edge = ren.GetSelectedShape()
tip_edge

<class 'TopoDS_Edge'>

In [23]:
# select tool shank edge  (inside edge)
shank_edge = ren.GetSelectedShape()
shank_edge

<class 'TopoDS_Edge'>

In [24]:
rm_chamfer_1 = HackerCAD.RuledMotion(tip_edge,shank_edge)
rm_chamfer_1.feedrate = 1000
rm_chamfer_1.surface_normal_offset = -25.4 / 16.0
rm_chamfer_1.tool_axis_offset = 1.0

In [25]:
# creating a lead in
lead_in_distance = -10.0
rm_chamfer_1.surface_normal_offset += lead_in_distance
v_tip_0,v_shank_0 = rm_chamfer_1.evaluate(0)

rm_chamfer_1.surface_normal_offset -= lead_in_distance
v_tip_1,v_shank_1 = rm_chamfer_1.evaluate(0)

me = BRepBuilderAPI_MakeEdge(
    gp_Pnt(v_tip_0.XYZ()),
    gp_Pnt(v_tip_1.XYZ()))
tip_lead_in_edge = me.Edge()

trsf = gp_Trsf()
trsf.SetTranslation(v_shank_0)

mt = BRepBuilderAPI_Transform(tip_lead_in_edge,trsf)
shank_lead_in_edge = mt.Shape()

In [26]:
lead_motion = HackerCAD.RuledMotion(tip_lead_in_edge,shank_lead_in_edge)
lead_motion.feedrate = 100

In [27]:
pp = HackerCAD.PostProcessor()

In [28]:
rm_gcode = pp.process([lead_motion,rm_chamfer_1])

In [29]:
gcode = ""
gcode += "G90 G21 G54\n"
gcode += "M6 T1 G43 H1\n"
gcode += "G93\n"
gcode += "M3 S21390\n"
gcode += rm_gcode
gcode += "G94\n"
gcode += "G91 G0 Y10\n"
gcode += "G90\n"
gcode += "G0 Z40\n"
gcode += "M5\n"
gcode += "M2\n"

In [30]:
f = open("/tmp/bla.ngc","w")
f.write(gcode)
f.close()

## OD chamfer, hex side

In [31]:
# select face to chamfer
chamfer_face = renderer.GetSelectedShape()
chamfer_face

<class 'TopoDS_Face'>

In [32]:
ren = JupyterRenderer()

In [33]:
for edge in OCCUtils.Topo(chamfer_face).edges():
    ren.DisplayShape(edge)

In [34]:
ren

In [35]:
# select tool tip edge (inside edge)
tip_edge = ren.GetSelectedShape()
tip_edge

<class 'TopoDS_Edge'>

In [36]:
# select tool shank edge  (outside edge)
shank_edge = ren.GetSelectedShape()
shank_edge

<class 'TopoDS_Edge'>

In [44]:
rm_chamfer_1 = HackerCAD.RuledMotion(tip_edge,shank_edge)
rm_chamfer_1.feedrate = 1000
rm_chamfer_1.surface_normal_offset = -25.4 / 16.0
rm_chamfer_1.tool_axis_offset = 0.4

In [45]:
# creating a lead in
lead_in_distance = -10.0
rm_chamfer_1.surface_normal_offset += lead_in_distance
v_tip_0,v_shank_0 = rm_chamfer_1.evaluate(0)

rm_chamfer_1.surface_normal_offset -= lead_in_distance
v_tip_1,v_shank_1 = rm_chamfer_1.evaluate(0)

me = BRepBuilderAPI_MakeEdge(
    gp_Pnt(v_tip_0.XYZ()),
    gp_Pnt(v_tip_1.XYZ()))
tip_lead_in_edge = me.Edge()

trsf = gp_Trsf()
trsf.SetTranslation(v_shank_0)

mt = BRepBuilderAPI_Transform(tip_lead_in_edge,trsf)
shank_lead_in_edge = mt.Shape()

In [46]:
lead_motion = HackerCAD.RuledMotion(tip_lead_in_edge,shank_lead_in_edge)
lead_motion.feedrate = 100

In [47]:
pp = HackerCAD.PostProcessor()

In [48]:
rm_gcode = pp.process([lead_motion,rm_chamfer_1])

In [51]:
gcode = ""
gcode += "G90 G21 G54\n"
gcode += "M6 T1 G43 H1\n"
gcode += "G93\n"
gcode += "M3 S21390\n"
gcode += rm_gcode
gcode += "G94\n"
gcode += "G91 G0 Y-3\n"
gcode += "G90\n"
gcode += "G0 Z40\n"
gcode += "M5\n"
gcode += "M2\n"

In [52]:
f = open("/tmp/bla.ngc","w")
f.write(gcode)
f.close()

# Thread

In [382]:
from math import sin,atan,pi
from OCC.Core.GCE2d import *
from OCC.Core.Geom import *

In [383]:
# select face to thread
thread_face = renderer.GetSelectedShape()
thread_face

<class 'TopoDS_Face'>

In [384]:
bas = BRepAdaptor_Surface(thread_face)

In [385]:
cyl = bas.Cylinder()

In [386]:
D_maj = cyl.Radius() * 2
D_maj

23.999999999999996

In [387]:
P = 1.5 # pitch

In [403]:
v_start = bas.FirstVParameter() - 1.5*P
v_end = bas.LastVParameter() + P
delta_v = v_end - v_start

In [404]:
dv_du = P / (2*pi)

In [405]:
l = delta_v / sin(atan(dv_du))

In [406]:
l2d = gp_Lin2d(gp_Pnt2d(0,v_start),
                       gp_Dir2d(1,dv_du))

In [407]:
#segment = GCE2d_MakeSegment(l2d,
#                            gp_Pnt2d(0,v_start),
#                            l)

In [408]:
segment = GCE2d_MakeSegment(l2d,0.1,l)

In [409]:
helix_edge = BRepBuilderAPI_MakeEdge(segment.Value(),Geom_CylindricalSurface(cyl)).Edge()

In [410]:
renderer.DisplayShape(helix_edge)

In [424]:
trsf = gp_Trsf()
trsf.SetTranslation(gp_Vec(0,1,0))
mt = BRepBuilderAPI_Transform(helix_edge,trsf)
helix_shank_edge = mt.Shape()

rm_thread = HackerCAD.RuledMotion(helix_edge,helix_shank_edge)

In [425]:
rm_thread.surface_normal_offset = - 0.140 * 25.4 / 2

In [426]:
rm_thread.feedrate = 1000

In [431]:
pp = HackerCAD.PostProcessor()

In [436]:
gcode_out = pp.process([rm_thread])

In [437]:
gcode = ""
gcode += "G90 G21 G54\n"
gcode += "M6 T1 G43 H1\n"
gcode += "G93\n"
gcode += "M3 S21390\n"
gcode +=  gcode_out
gcode += "G94\n"
gcode += "G91 G0 Y-3\n"
gcode += "G90\n"
gcode += "G0 Z40\n"
gcode += "M5\n"
gcode += "M2\n"

In [438]:
f = open("/tmp/bla.ngc","w")
f.write(gcode)
f.close()

In [15]:
a = [1,2,3,4]

In [16]:
a[1:]

[2, 3, 4]

## Cutoff

In [19]:
# select cutoff face
chamfer_face = renderer.GetSelectedShape()
chamfer_face

<class 'TopoDS_Face'>

In [20]:
ren = JupyterRenderer()

In [21]:
for edge in OCCUtils.Topo(chamfer_face).edges():
    ren.DisplayShape(edge)

In [22]:
ren

In [23]:
# select the circular edge
tip_edge = ren.GetSelectedShape()
tip_edge

<class 'TopoDS_Edge'>

In [24]:
# create a tool shank edge
bac = BRepAdaptor_Curve(tip_edge)
circ = bac.Circle()
ax = gp_Ax2(circ.Location(),circ.Axis().Direction(),circ.XAxis().Direction())
new_circ = gp_Circ(ax,circ.Radius()+1)
me = BRepBuilderAPI_MakeEdge(new_circ)
shank_edge = me.Edge()

In [25]:
circ.Axis()

< gp_Ax1: location: 0.0, 14.643000000000004, 3.330669073875464e-15, direction: 0.0, -1.0, 2.22044604925031e-16 >

In [33]:
tool_diameter = 25.4 / 16.0

rm_cutoff = HackerCAD.RuledMotion(tip_edge,shank_edge)
rm_cutoff.feedrate = 1000

rm_cutoff.use_trochoidal_offset = True
rm_cutoff.trochoidal_x = tool_diameter / 2
rm_cutoff.trochoidal_y = tool_diameter / 4
rm_cutoff.trochoidal_period = 0.02

rm_cutoff.surface_normal_offset = - (tool_diameter / 2 + rm_cutoff.trochoidal_x)
rm_cutoff.tool_axis_offset = 0.4

In [34]:
pp = HackerCAD.PostProcessor()

In [35]:
rm_gcode = pp.process([rm_cutoff])

In [36]:
gcode = ""
gcode += "G90 G21 G54\n"
gcode += "M6 T1 G43 H1\n"
gcode += "G93\n"
gcode += "M3 S21390\n"
gcode += rm_gcode
gcode += "G94\n"
gcode += "G91 G0 Y5\n"
gcode += "G90\n"
gcode += "G0 Z40\n"
gcode += "M5\n"
gcode += "M2\n"

In [37]:
f = open("/tmp/bla.ngc","w")
f.write(gcode)
f.close()